In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50 
import pickle

In [2]:
with open('train_img.pkl', 'rb') as file:
    X = pickle.load(file)

In [3]:
with open('train_label_modified.pkl', 'rb') as file:
    Y = pickle.load(file)

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

In [5]:
base_model.trainable = False

In [6]:
model = tf.keras.Sequential([
    base_model,  
    tf.keras.layers.Flatten(),  
    tf.keras.layers.Dense(200, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)),  
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.001)), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation="softmax"),  
])

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │    20,070,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           303 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,678,715 (166.62 MB)

 Trainable params: 20,091,003 (76.64 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [9]:
h = model.fit(
    x=X,
    y=Y,
    epochs=20,
    validation_split=0.2,
    batch_size=8,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)

Epoch 1/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 106s 368ms/step - accuracy: 0.4415 - loss: 1.6136 - val_accuracy: 0.9057 - val_loss: 0.8411
Epoch 2/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 110s 390ms/step - accuracy: 0.5323 - loss: 0.9595 - val_accuracy: 0.7206 - val_loss: 0.6792
Epoch 3/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 111s 395ms/step - accuracy: 0.5969 - loss: 0.8518 - val_accuracy: 0.7206 - val_loss: 0.6259
Epoch 4/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 113s 400ms/step - accuracy: 0.6020 - loss: 0.8276 - val_accuracy: 0.7206 - val_loss: 0.6528
Epoch 5/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 120s 426ms/step - accuracy: 0.6023 - loss: 0.8094 - val_accuracy: 1.0000 - val_loss: 0.5500
Epoch 6/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 110s 392ms/step - accuracy: 0.6617 - loss: 0.7680 - val_accuracy: 1.0000 - val_loss: 0.4993
Epoch 7/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 112s 400ms/step - accuracy: 0.6847 - loss: 0.7268 - val_accuracy: 1.0000 - val_loss: 0.5077
Epoch 8/20
281/281 ━━━━━━━━━━━━━━━━━━━━ 119s 425ms/step - accuracy: 0.7456 -

In [10]:
model.save("final_model_with_pretrained_model.h5")